In [1]:
import pandas as pd
import math
import numpy as np

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "valid_log.csv").is_file():
        cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
        cfg = OmegaConf.to_container(cfg)
        num_epochs = cfg['optim']['epochs']

        valid_log = pd.read_csv(run / "valid_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-3) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    len(list(Path('.').glob('*')))
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def check_image_size(run, image_size=480):
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)
    conf_image_size = cfg['data']['image_size']
    if int(conf_image_size) != image_size:
        print("Different image size in conf. Image size in conf: {}, Image size: {}".format(conf_image_size, image_size))

def collect_one(model_name, run, csv_file, image_size=480):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    check_image_size(run, image_size=image_size)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file, regimes=['0.01', '0.02', '0.05', '0.1', '0.2', '0.25', '1.0'], image_size=480):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    metrics.append(collect_one(model_name, run, csv_file, image_size=image_size))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

def collect_all_temperature(model_name, root, csv_file, regimes=['0.01', '0.02', '0.05', '0.1', '0.2', '0.25', '1.0'], image_size=480):
    root = Path(root)

    metrics = []
    for regime in list(root.glob("regime-*")):
        if regime.name.rsplit("-", 1)[1] in regimes:
            for run in list(regime.glob("run-*")):
                metrics.append(collect_one(model_name, run, csv_file, image_size=image_size))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90):

    def compute_ci(values):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        return ci

    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
                
    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [18]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['1.0']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t')),
}

In [19]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [20]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,52.31,36.16,41.72,8.55
1,H-UNet-BASE-SWTA,0,5.0,1.0,49.74,33.87,42.40,8.16
2,H-UNet-BASE-SWTA,0,10.0,1.0,45.79,30.24,45.59,8.88
3,H-UNet-BASE-SWTA,0,20.0,1.0,49.81,33.74,38.32,7.20
4,H-UNet-BASE-SWTA,0,50.0,1.0,49.98,34.01,42.11,7.88
5,H-UNet-BASE-SWTA,0,100.0,1.0,49.48,33.46,40.05,7.27
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,45.60,30.22,44.54,9.98
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,48.07,32.26,54.12,10.52
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,49.61,33.60,42.02,8.29
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,50.11,34.03,42.04,7.84


/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600dde10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function std at 0x7faf600ddfc0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'std' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastruc

Dice                 Jaccard      \
                                     Mean STD     CI 0.9%    Mean STD   
Model              Inv Temp Regime                                      
H-UNet-BASE-SWTA   1.0      1.0     52.31 NaN  (nan, nan)   36.16 NaN   
                   5.0      1.0     49.74 NaN  (nan, nan)   33.87 NaN   
                   10.0     1.0     45.79 NaN  (nan, nan)   30.24 NaN   
                   20.0     1.0     49.81 NaN  (nan, nan)   33.74 NaN   
                   50.0     1.0     49.98 NaN  (nan, nan)   34.01 NaN   
                   100.0    1.0     49.48 NaN  (nan, nan)   33.46 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     45.60 NaN  (nan, nan)   30.22 NaN   
                   5.0      1.0     48.07 NaN  (nan, nan)   32.26 NaN   
                   10.0     1.0     49.61 NaN  (nan, nan)   33.60 NaN   
                   20.0     1.0     50.11 NaN  (nan, nan)   34.03 NaN   
                   50.0     1.0     47.67 NaN  (nan, nan)   31.89 NaN   
                   100.0    1.0     49.24 NaN  (nan, nan)   33.43 NaN   

                                               Hausdorff Distance      \
                                       CI 0.9%               Mean STD   
Model              Inv Temp Regime                                      
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)              41.72 NaN   
                   5.0      1.0     (nan, nan)              42.40 NaN   
                   10.0     1.0     (nan, nan)              45.59 NaN   
                   20.0     1.0     (nan, nan)              38.32 NaN   
                   50.0     1.0     (nan, nan)              42.11 NaN   
                   100.0    1.0     (nan, nan)              40.05 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)              44.54 NaN   
                   5.0      1.0     (nan, nan)              54.12 NaN   
                   10.0     1.0     (nan, nan)              42.02 NaN   
                   20.0     1.0     (nan, nan)              42.04 NaN   
                   50.0     1.0     (nan, nan)              43.38 NaN   
                   100.0    1.0     (nan, nan)              41.65 NaN   

                                               Average Surface Distance      \
                                       CI 0.9%                     Mean STD   
Model              Inv Temp Regime                                            
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)                     8.55 NaN   
                   5.0      1.0     (nan, nan)                     8.16 NaN   
                   10.0     1.0     (nan, nan)                     8.88 NaN   
                   20.0     1.0     (nan, nan)                     7.20 NaN   
                   50.0     1.0     (nan, nan)                     7.88 NaN   
                   100.0    1.0     (nan, nan)                     7.27 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)                     9.98 NaN   
                   5.0      1.0     (nan, nan)                    10.52 NaN   
                   10.0     1.0     (nan, nan)                     8.29 NaN   
                   20.0     1.0     (nan, nan)                     7.84 NaN   
                   50.0     1.0     (nan, nan)                     8.20 NaN   
                   100.0    1.0     (nan, nan)                     8.29 NaN   

                                                
                                       CI 0.9%  
Model              Inv Temp Regime              
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)  
                   5.0      1.0     (nan, nan)  
                   10.0     1.0     (nan, nan)  
                   20.0     1.0     (nan, nan)  
                   50.0     1.0     (nan, nan)  
                   100.0    1.0     (nan, nan)  
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)  
                   5.0      1.0     (nan, nan)  
                   10.0     1.0     (nan, nan)  
                   20.0     1.0     (nan, nan)  
                   50.

In [21]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_ft')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t_ft')),
}

In [22]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)

In [23]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT,0,1.0,0.2,78.29,65.23,25.62,3.60
1,H-UNet-BASE-SWTA-FT,0,5.0,0.2,79.96,67.77,27.76,3.96
2,H-UNet-BASE-SWTA-FT,0,10.0,0.2,81.71,70.05,23.32,3.45
3,H-UNet-BASE-SWTA-FT,0,20.0,0.2,81.64,69.71,19.47,2.40
4,H-UNet-BASE-SWTA-FT,0,50.0,0.2,81.50,69.69,22.19,3.04
...,...,...,...,...,...,...,...,...
115,H-UNet-BASE-SWTA-T-FT,9,5.0,0.2,80.07,68.01,22.94,3.58
116,H-UNet-BASE-SWTA-T-FT,9,10.0,0.2,79.62,67.11,26.51,3.54
117,H-UNet-BASE-SWTA-T-FT,9,20.0,0.2,80.61,68.63,28.82,3.85
118,H-UNet-BASE-SWTA-T-FT,9,50.0,0.2,79.22,66.88,27.03,3.87


/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600dde10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function std at 0x7faf600ddfc0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'std' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600d

Dice            \
                                         Mean       STD   
Model                 Inv Temp Regime                     
H-UNet-BASE-SWTA-FT   1.0      0.2     77.890  1.997649   
                      5.0      0.2     79.583  1.966842   
                      10.0     0.2     79.743  1.593606   
                      20.0     0.2     80.649  1.280663   
                      50.0     0.2     80.322  1.563833   
                      100.0    0.2     80.417  2.325621   
H-UNet-BASE-SWTA-T-FT 1.0      0.2     78.210  1.914233   
                      5.0      0.2     80.459  1.786520   
                      10.0     0.2     81.042  1.461079   
                      20.0     0.2     81.511  2.092566   
                      50.0     0.2     81.150  1.887467   
                      100.0    0.2     80.874  2.195269   

                                                                               \
                                                                      CI 0.9%   
Model                 Inv Temp Regime                                           
H-UNet-BASE-SWTA-FT   1.0      0.2     (76.79142534517122, 78.98857465482875)   
                      5.0      0.2     (78.50136690258473, 80.66463309741526)   
                      10.0     0.2     (78.86662227563097, 80.61937772436904)   
                      20.0     0.2     (79.94471977663792, 81.35328022336208)   
                      50.0     0.2      (79.4619953069681, 81.18200469303187)   
                      100.0    0.2     (79.13806231670831, 81.69593768329169)   
H-UNet-BASE-SWTA-T-FT 1.0      0.2     (77.15729828867464, 79.26270171132538)   
                      5.0      0.2      (79.47653201208844, 81.4414679879116)   
                      10.0     0.2     (80.23850329481239, 81.84549670518761)   
                      20.0     0.2     (80.36022709270742, 82.66177290729257)   
                      50.0     0.2     (80.11201770827493, 82.18798229172508)   
                      100.0    0.2     (79.66674739426463, 82.08125260573536)   

                                      Jaccard            \
                                         Mean       STD   
Model                 Inv Temp Regime                     
H-UNet-BASE-SWTA-FT   1.0      0.2     64.949  2.580833   
                      5.0      0.2     67.351  2.601664   
                      10.0     0.2     67.467  2.118732   
                      20.0     0.2     68.642  1.708799   
                      50.0     0.2     68.282  2.008149   
                      100.0    0.2     68.408  3.034636   
H-UNet-BASE-SWTA-T-FT 1.0      0.2     65.403  2.549466   
                      5.0      0.2     68.528  2.441210   
                      10.0     0.2     69.245  1.992058   
                      20.0     0.2     69.898  2.830684   
                      50.0     0.2     69.536  2.490543   
                      100.0    0.2     69.032  2.904953   

                                                                               \
                                                                      CI 0.9%   
Model                 Inv Temp Regime                                           
H-UNet-BASE-SWTA-FT   1.0      0.2      (63.5297125042603, 66.36828749573972)   
                      5.0      0.2     (65.92025694139619, 68.78174305860378)   
                      10.0     0.2     (66.30183774464376, 68.63216225535623)   
                      20.0     0.2     (67.70227328943477, 69.58172671056522)   
                      50.0     0.2     (67.17765085441276, 69.38634914558723)   
                      100.0    0.2     (66.73915065818034, 70.07684934181964)   
H-UNet-BASE-SWTA-T-FT 1.0      0.2     (64.00096205510367, 66.80503794489631)   
                      5.0      0.2     (67.18549586795967, 69.87050413204031)   
                      10.0     0.2         (68.149499902368, 70.340500097632)   
                      20.0     0.2     (68.34131082693528, 71.45468917306471)   


<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [24]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

IMAGE_SIZE = 480
INV_TEMP_GlaS=1          # to be set accordingly, used by SWTA

runs = {
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
}

In [25]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv', image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [26]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_741713/3978195719.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,52.31,36.16,41.72,8.55
1,H-UNet-BASE-SWTA-T,0,1.0,1.0,45.60,30.22,44.54,9.98
2,H-UNet-HPCA,0,1.0,1.0,33.72,20.47,NaN,NaN
3,H-UNet-HPCA-T,0,1.0,1.0,34.84,21.30,NaN,NaN


/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600dde10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function std at 0x7faf600ddfc0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'std' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastruc

Dice                 Jaccard      \
                                     Mean STD     CI 0.9%    Mean STD   
Model              Inv Temp Regime                                      
H-UNet-BASE-SWTA   1.0      1.0     52.31 NaN  (nan, nan)   36.16 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     45.60 NaN  (nan, nan)   30.22 NaN   
H-UNet-HPCA        1.0      1.0     33.72 NaN  (nan, nan)   20.47 NaN   
H-UNet-HPCA-T      1.0      1.0     34.84 NaN  (nan, nan)   21.30 NaN   

                                               Hausdorff Distance      \
                                       CI 0.9%               Mean STD   
Model              Inv Temp Regime                                      
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)              41.72 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)              44.54 NaN   
H-UNet-HPCA        1.0      1.0     (nan, nan)                NaN NaN   
H-UNet-HPCA-T      1.0      1.0     (nan, nan)                NaN NaN   

                                               Average Surface Distance      \
                                       CI 0.9%                     Mean STD   
Model              Inv Temp Regime                                            
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)                     8.55 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)                     9.98 NaN   
H-UNet-HPCA        1.0      1.0     (nan, nan)                      NaN NaN   
H-UNet-HPCA-T      1.0      1.0     (nan, nan)                      NaN NaN   

                                                
                                       CI 0.9%  
Model              Inv Temp Regime              
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)  
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)  
H-UNet-HPCA        1.0      1.0     (nan, nan)  
H-UNet-HPCA-T      1.0      1.0     (nan, nan)

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [27]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2']       # regimes to be considered
IMAGE_SIZE = 480
INV_TEMP_GlaS=20          # to be set accordingly, used by SWTA

runs = {
    'UNet': list(Path(EXP_ROOT + '/experiment=glas/').glob('unet_base')),
    'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=glas').glob('unet')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-hpca_ft')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-hpca_t_ft')),
    'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas').glob('hunet-hpca_ft')),
    'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas').glob('hunet-hpca_t_ft')),
}

runs_swta = {
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft').glob('inv_temp-{}'.format(INV_TEMP_GlaS))),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft').glob('inv_temp-{}'.format(INV_TEMP_GlaS))),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
}

In [28]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
if bool([a for a in runs_swta.values() if a != []]):
    predictions_swta = pd.concat([collect_all_temperature(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs_swta.items() for r in v], ignore_index=True)
    predictions = pd.concat([predictions, predictions_swta])

In [29]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics, confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA-FT,0,1.0,0.01,67.58,52.73,58.18,8.92
1,H-UNet-HPCA-FT,0,1.0,0.02,67.91,52.48,57.22,7.73
2,H-UNet-HPCA-FT,0,1.0,0.05,68.29,52.91,38.88,5.62
3,H-UNet-HPCA-FT,0,1.0,0.10,75.15,61.12,26.50,4.05
4,H-UNet-HPCA-FT,0,1.0,0.20,78.65,65.73,26.27,3.62
...,...,...,...,...,...,...,...,...
315,UNet-Pseudolabeling,9,1.0,0.01,67.13,52.14,59.80,8.74
316,UNet-Pseudolabeling,9,1.0,0.02,66.40,51.12,61.72,9.10
317,UNet-Pseudolabeling,9,1.0,0.05,65.88,50.97,61.57,9.49
318,UNet-Pseudolabeling,9,1.0,0.10,66.22,50.99,59.10,8.89


/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600dde10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function std at 0x7faf600ddfc0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'std' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600d

Dice            \
                                                   Mean       STD   
Model                           Inv Temp Regime                     
H-UNet-HPCA-FT                  1.0      0.01    65.402  2.947944   
                                         0.02    66.603  3.171295   
                                         0.05    71.776  2.843840   
                                         0.10    76.394  2.208781   
                                         0.20    79.593  1.425997   
H-UNet-HPCA-T-FT                1.0      0.01    64.482  6.466331   
                                         0.02    67.391  2.127061   
                                         0.05    70.056  4.095038   
                                         0.10    76.170  2.022765   
                                         0.20    79.974  1.944258   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.01    57.611  6.553048   
                                         0.02    63.048  7.210580   
                                         0.05    69.515  2.848494   
                                         0.10    71.930  5.291977   
                                         0.20    78.046  4.778073   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.01    58.841  8.462337   
                                         0.02    64.060  7.720245   
                                         0.05    67.878  4.539784   
                                         0.10    72.655  3.239380   
                                         0.20    78.723  5.588059   
H-UNet-SWTA-FT                  20.0     0.20    80.649  1.280663   
H-UNet-SWTA-T-FT                20.0     0.20    81.511  2.092566   
UNet                            1.0      0.01    66.862  2.429366   
                                         0.02    67.843  2.596108   
                                         0.05    69.953  4.903072   
                                         0.10    74.390  2.723148   
                                         0.20    79.676  2.017701   
UNet-Pseudolabeling             1.0      0.01    64.625  4.539178   
                                         0.02    66.907  1.365463   
                                         0.05    66.717  3.789374   
                                         0.10    70.947  5.110051   
                                         0.20    81.449  2.045317   

                                                                                           \
                                                                                  CI 0.9%   
Model                           Inv Temp Regime                                             
H-UNet-HPCA-FT                  1.0      0.01       (63.7808257647174, 67.02317423528257)   
                                         0.02      (64.85899741541225, 68.34700258458774)   
                                         0.05      (70.21207596556175, 73.33992403443824)   
                                         0.10      (75.17931623990002, 77.60868376009999)   
                                         0.20      (78.80879587285366, 80.37720412714631)   
H-UNet-HPCA-T-FT                1.0      0.01        (60.9259453606129, 68.0380546393871)   
                                         0.02       (66.2212571847747, 68.56074281522528)   
                                         0.05      (67.80399976024543, 72.30800023975459)   
                                         0.10      (75.05761305917726, 77.28238694082275)   
                                         0.20       (78.90478685171041, 81.0432131482896)   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.01      (54.00725713509565, 61.21474286490436)   
                                         0.02      (59.08265783038805, 67.01334216961195)   
                                         0.05      (67.94851677827918, 71.08148322172079)   
                                         0.10      (69.01976247067373, 74.84023752932626)   
                                         0.20      (75

<h1>PH2 Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [4]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['1.0']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_t')),
}

In [5]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [6]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,38.46,24.57,88.37,19.27
1,H-UNet-BASE-SWTA,0,5.0,1.0,48.69,33.11,128.40,17.75
2,H-UNet-BASE-SWTA,0,10.0,1.0,41.50,26.80,128.04,21.26
3,H-UNet-BASE-SWTA,0,20.0,1.0,50.86,35.08,118.39,16.46
4,H-UNet-BASE-SWTA,0,50.0,1.0,51.85,36.25,67.03,10.53
5,H-UNet-BASE-SWTA,0,100.0,1.0,49.54,34.06,128.33,18.33
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,39.18,25.08,142.65,29.84
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,47.29,31.72,128.85,18.26
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,51.08,35.17,118.49,16.77
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,45.40,30.08,128.75,20.37


/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600dde10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function std at 0x7faf600ddfc0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'std' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastruc

Dice                 Jaccard      \
                                     Mean STD     CI 0.9%    Mean STD   
Model              Inv Temp Regime                                      
H-UNet-BASE-SWTA   1.0      1.0     38.46 NaN  (nan, nan)   24.57 NaN   
                   5.0      1.0     48.69 NaN  (nan, nan)   33.11 NaN   
                   10.0     1.0     41.50 NaN  (nan, nan)   26.80 NaN   
                   20.0     1.0     50.86 NaN  (nan, nan)   35.08 NaN   
                   50.0     1.0     51.85 NaN  (nan, nan)   36.25 NaN   
                   100.0    1.0     49.54 NaN  (nan, nan)   34.06 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     39.18 NaN  (nan, nan)   25.08 NaN   
                   5.0      1.0     47.29 NaN  (nan, nan)   31.72 NaN   
                   10.0     1.0     51.08 NaN  (nan, nan)   35.17 NaN   
                   20.0     1.0     45.40 NaN  (nan, nan)   30.08 NaN   
                   50.0     1.0     51.11 NaN  (nan, nan)   35.35 NaN   
                   100.0    1.0     53.49 NaN  (nan, nan)   37.81 NaN   

                                               Hausdorff Distance      \
                                       CI 0.9%               Mean STD   
Model              Inv Temp Regime                                      
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)              88.37 NaN   
                   5.0      1.0     (nan, nan)             128.40 NaN   
                   10.0     1.0     (nan, nan)             128.04 NaN   
                   20.0     1.0     (nan, nan)             118.39 NaN   
                   50.0     1.0     (nan, nan)              67.03 NaN   
                   100.0    1.0     (nan, nan)             128.33 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)             142.65 NaN   
                   5.0      1.0     (nan, nan)             128.85 NaN   
                   10.0     1.0     (nan, nan)             118.49 NaN   
                   20.0     1.0     (nan, nan)             128.75 NaN   
                   50.0     1.0     (nan, nan)             108.32 NaN   
                   100.0    1.0     (nan, nan)              92.96 NaN   

                                               Average Surface Distance      \
                                       CI 0.9%                     Mean STD   
Model              Inv Temp Regime                                            
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)                    19.27 NaN   
                   5.0      1.0     (nan, nan)                    17.75 NaN   
                   10.0     1.0     (nan, nan)                    21.26 NaN   
                   20.0     1.0     (nan, nan)                    16.46 NaN   
                   50.0     1.0     (nan, nan)                    10.53 NaN   
                   100.0    1.0     (nan, nan)                    18.33 NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)                    29.84 NaN   
                   5.0      1.0     (nan, nan)                    18.26 NaN   
                   10.0     1.0     (nan, nan)                    16.77 NaN   
                   20.0     1.0     (nan, nan)                    20.37 NaN   
                   50.0     1.0     (nan, nan)                    14.29 NaN   
                   100.0    1.0     (nan, nan)                    12.72 NaN   

                                                
                                       CI 0.9%  
Model              Inv Temp Regime              
H-UNet-BASE-SWTA   1.0      1.0     (nan, nan)  
                   5.0      1.0     (nan, nan)  
                   10.0     1.0     (nan, nan)  
                   20.0     1.0     (nan, nan)  
                   50.0     1.0     (nan, nan)  
                   100.0    1.0     (nan, nan)  
H-UNet-BASE-SWTA-T 1.0      1.0     (nan, nan)  
                   5.0      1.0     (nan, nan)  
                   10.0     1.0     (nan, nan)  
                   20.0     1.0     (nan, nan)  
                   50.

In [10]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t_ft')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [12]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

IMAGE_SIZE = 480
INV_TEMP_PH2=100          # to be set accordingly, used by SWTA

runs = {
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run-*')),
}

In [13]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv', image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [14]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_741713/3978195719.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,100.0,1.0,49.54,34.06,128.33,18.33
1,H-UNet-BASE-SWTA-T,0,100.0,1.0,53.49,37.81,92.96,12.72
2,H-UNet-HPCA,0,1.0,1.0,21.33,12.19,246.21,86.46
3,H-UNet-HPCA-T,0,1.0,1.0,21.98,12.62,NaN,NaN


/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600dde10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function std at 0x7faf600ddfc0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'std' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastruc

Dice                 Jaccard      \
                                     Mean STD     CI 0.9%    Mean STD   
Model              Inv Temp Regime                                      
H-UNet-BASE-SWTA   100.0    1.0     49.54 NaN  (nan, nan)   34.06 NaN   
H-UNet-BASE-SWTA-T 100.0    1.0     53.49 NaN  (nan, nan)   37.81 NaN   
H-UNet-HPCA        1.0      1.0     21.33 NaN  (nan, nan)   12.19 NaN   
H-UNet-HPCA-T      1.0      1.0     21.98 NaN  (nan, nan)   12.62 NaN   

                                               Hausdorff Distance      \
                                       CI 0.9%               Mean STD   
Model              Inv Temp Regime                                      
H-UNet-BASE-SWTA   100.0    1.0     (nan, nan)             128.33 NaN   
H-UNet-BASE-SWTA-T 100.0    1.0     (nan, nan)              92.96 NaN   
H-UNet-HPCA        1.0      1.0     (nan, nan)             246.21 NaN   
H-UNet-HPCA-T      1.0      1.0     (nan, nan)                NaN NaN   

                                               Average Surface Distance      \
                                       CI 0.9%                     Mean STD   
Model              Inv Temp Regime                                            
H-UNet-BASE-SWTA   100.0    1.0     (nan, nan)                    18.33 NaN   
H-UNet-BASE-SWTA-T 100.0    1.0     (nan, nan)                    12.72 NaN   
H-UNet-HPCA        1.0      1.0     (nan, nan)                    86.46 NaN   
H-UNet-HPCA-T      1.0      1.0     (nan, nan)                      NaN NaN   

                                                
                                       CI 0.9%  
Model              Inv Temp Regime              
H-UNet-BASE-SWTA   100.0    1.0     (nan, nan)  
H-UNet-BASE-SWTA-T 100.0    1.0     (nan, nan)  
H-UNet-HPCA        1.0      1.0     (nan, nan)  
H-UNet-HPCA-T      1.0      1.0     (nan, nan)

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [15]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2']       # regimes to be considered
IMAGE_SIZE = 480
INV_TEMP_PH2=20          # to be set accordingly, used by SWTA

runs = {
    'UNet': list(Path(EXP_ROOT + '/experiment=ph2/').glob('unet_base')),
    'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=ph2').glob('unet')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-hpca_ft')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-hpca_t_ft')),
    'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2').glob('hunet-hpca_ft')),
    'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2').glob('hunet-hpca_t_ft')),
}

runs_swta = {
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_ft').glob('inv_temp-{}'.format(INV_TEMP_PH2))),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_t_ft').glob('inv_temp-{}'.format(INV_TEMP_PH2))),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
}

In [16]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
if bool([a for a in runs_swta.values() if a != []]):
    predictions_swta = pd.concat([collect_all_temperature(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs_swta.items() for r in v], ignore_index=True)
    predictions = pd.concat([predictions, predictions_swta])

In [17]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA-FT,0,1.0,0.01,62.41,49.19,63.41,14.78
1,H-UNet-HPCA-FT,0,1.0,0.02,79.86,68.31,12.28,1.90
2,H-UNet-HPCA-FT,0,1.0,0.05,75.04,61.78,38.92,6.86
3,H-UNet-HPCA-FT,0,1.0,0.10,85.94,76.63,11.83,1.69
4,H-UNet-HPCA-FT,0,1.0,0.20,87.72,78.98,9.28,1.27
...,...,...,...,...,...,...,...,...
295,UNet-Pseudolabeling,9,1.0,0.01,61.79,46.94,69.47,11.93
296,UNet-Pseudolabeling,9,1.0,0.02,69.18,57.83,74.87,11.71
297,UNet-Pseudolabeling,9,1.0,0.05,70.11,57.25,111.20,16.01
298,UNet-Pseudolabeling,9,1.0,0.10,83.72,74.73,20.54,3.75


/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600dde10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function std at 0x7faf600ddfc0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'std' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_741713/3978195719.py:46: FutureWarning: The provided callable <function mean at 0x7faf600d

Dice             \
                                                   Mean        STD   
Model                           Inv Temp Regime                      
H-UNet-HPCA-FT                  1.0      0.01    72.647   5.085620   
                                         0.02    69.030  10.023519   
                                         0.05    77.375   4.425128   
                                         0.10    82.307   3.323546   
                                         0.20    86.187   2.229205   
H-UNet-HPCA-T-FT                1.0      0.01    63.658   9.259468   
                                         0.02    72.936   6.932417   
                                         0.05    74.626   2.971592   
                                         0.10    80.270   4.130297   
                                         0.20    85.736   3.368017   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.01    60.290  17.303537   
                                         0.02    67.078  17.837791   
                                         0.05    81.676   3.056149   
                                         0.10    85.712   3.547249   
                                         0.20    90.117   2.446290   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.01    63.573  15.037125   
                                         0.02    75.012   6.618032   
                                         0.05    82.633   4.546794   
                                         0.10    85.136   2.876928   
                                         0.20    89.927   1.635917   
UNet                            1.0      0.01    73.764   5.324234   
                                         0.02    74.482   6.478708   
                                         0.05    77.064   3.952212   
                                         0.10    80.056   3.879960   
                                         0.20    84.177   2.320656   
UNet-Pseudolabeling             1.0      0.01    65.807  11.815151   
                                         0.02    71.648  13.812952   
                                         0.05    80.633   5.769306   
                                         0.10    86.007   3.269900   
                                         0.20    89.660   1.770273   

                                                                                          \
                                                                                 CI 0.9%   
Model                           Inv Temp Regime                                            
H-UNet-HPCA-FT                  1.0      0.01     (69.85024506434183, 75.44375493565818)   
                                         0.02     (63.51772730190882, 74.54227269809118)   
                                         0.05      (74.94147198563917, 79.8085280143608)   
                                         0.10     (80.47926956501774, 84.13473043498223)   
                                         0.20      (84.9610844315254, 87.41291556847459)   
H-UNet-HPCA-T-FT                1.0      0.01     (58.56590473492478, 68.75009526507523)   
                                         0.02      (69.12362906318148, 76.7483709368185)   
                                         0.05       (72.9918207910755, 76.2601792089245)   
                                         0.10     (77.99860959646689, 82.54139040353313)   
                                         0.20     (83.88381340393674, 87.58818659606324)   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.01     (50.77419900068554, 69.80580099931447)   
                                         0.02     (57.26839426820596, 76.88760573179404)   
                                         0.05     (79.99531992717482, 83.35668007282521)   
                                         0.10     (83.76124776029715, 87.66275223970288)   
                                         0.20     (88.77170228512342, 91.46229771487658)   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.01     (55.30357539478612, 71.84242460521